In [1]:
from gensim.models import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
wv = KeyedVectors.load_word2vec_format('glove.6B.300d.bin', binary=True)
annoy_index = AnnoyIndexer()
annoy_index.load('glove.6B.300d.index')
annoy_index.model = wv

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
wv.most_similar_to_given('disaster',['crash', 'tragedy', 'accidents', 'disasters', 'collision', 'catastrophe', 'happened', 'occurred'])

'catastrophe'

In [3]:
vocab_disasters_and_accdients = dict(wv.most_similar(positive=['disaster','accident',], topn=1000, indexer=annoy_index))

In [4]:
vocab_disasters = dict(wv.most_similar(positive=['disaster'], topn=1000, indexer=annoy_index))

In [5]:
vocab_accdients = dict(wv.most_similar(positive=['accident'], topn=1000, indexer=annoy_index))

In [6]:
list(vocab_disasters_and_accdients.items())[-10:]

[('passing', 0.39112114906311035),
 ('place', 0.3911011219024658),
 ('lasted', 0.3910970687866211),
 ('beginning', 0.39105361700057983),
 ('sunk', 0.3910179138183594),
 ('supposed', 0.39100825786590576),
 ('dangers', 0.3909822106361389),
 ('plans', 0.3909767270088196),
 ('anniversary', 0.39094823598861694),
 ('force', 0.39094412326812744)]

In [8]:
list(vocab_disasters.items())[-10:]

[('force', 0.3822338581085205),
 ('they', 0.38219934701919556),
 ('visit', 0.38216954469680786),
 ('airline', 0.38215988874435425),
 ('big', 0.38210928440093994),
 ('crucial', 0.3820965886116028),
 ('adequate', 0.38208138942718506),
 ('southern', 0.38207387924194336),
 ('part', 0.3820651173591614),
 ('campaign', 0.3820647597312927)]

In [9]:
list(vocab_accdients.items())[-10:]

[('august', 0.38355422019958496),
 ('closure', 0.38352274894714355),
 ('described', 0.38349199295043945),
 ('claiming', 0.3834758996963501),
 ('bizarre', 0.3834714889526367),
 ('less', 0.3834567666053772),
 ('horizon', 0.38344794511795044),
 ('liability', 0.38336771726608276),
 ('brief', 0.38336360454559326),
 ('infant', 0.38335514068603516)]

In [10]:
vocab = list(vocab_disasters_and_accdients.keys())

In [18]:
len(vocab)

1000

In [11]:
vocab_ = list(set(list(vocab_disasters.keys()) + list(vocab_accdients.keys())))

In [19]:
len(vocab_)

1613

In [17]:
len(set(vocab) & set(vocab_))

971

In [20]:
vocab = vocab_

In [21]:
from uri import *

In [22]:
uri_to_label('/c/en/24_hour_clock/n')

'24 hour clock'

In [23]:
import dask.dataframe as dd

In [24]:
df_conceptnet = dd.read_csv('./ccnet/neo-en-noURL/conceptnet-rel-*.csv',names=['start','rel','end','weight'])

In [25]:
df_conceptnet['start_label'] = df_conceptnet['start'].apply(uri_to_label)
df_conceptnet['end_label'] = df_conceptnet['end'].apply(uri_to_label)

D:\Anaconda3\lib\site-packages\dask\dataframe\core.py:2184: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [26]:
df_conceptnet.head()

,start,rel,end,weight,start_label,end_label
0,/c/en/0/n,/r/Antonym,/c/en/1,1.0,0,1
1,/c/en/12_hour_clock/n,/r/Antonym,/c/en/24_hour_clock,1.0,12 hour clock,24 hour clock
2,/c/en/24_hour_clock/n,/r/Antonym,/c/en/12_hour_clock,1.0,24 hour clock,12 hour clock
3,/c/en/5/n,/r/Antonym,/c/en/3,1.0,5,3
4,/c/en/a.c/n,/r/Antonym,/c/en/d.c,1.0,a.c,d.c


In [27]:
df_disaster = dd.concat([df_conceptnet[df_conceptnet.start_label.isin(vocab)],df_conceptnet[df_conceptnet.end_label.isin(vocab)]])

In [28]:
df_disaster.head()

,start,rel,end,weight,start_label,end_label
59,/c/en/about,/r/Antonym,/c/en/precisely,0.155,about,precisely
132,/c/en/accident,/r/Antonym,/c/en/on_purpose,0.319,accident,on purpose
133,/c/en/accident,/r/Antonym,/c/en/planned,0.147,accident,planned
134,/c/en/accident,/r/Antonym,/c/en/purpose,0.319,accident,purpose
135,/c/en/accidentally/r,/r/Antonym,/c/en/intentionally,1.000,accidentally,intentionally


In [29]:
df_disaster.tail()

,start,rel,end,weight,start_label,end_label
51982,/c/en/victoria/n,/r/dbpedia/product,/c/en/motorcycle,0.5,victoria,motorcycle
51983,/c/en/volkswagen,/r/dbpedia/product,/c/en/bus,0.5,volkswagen,bus
51984,/c/en/volkswagen,/r/dbpedia/product,/c/en/car,0.5,volkswagen,car
51985,/c/en/volkswagen,/r/dbpedia/product,/c/en/truck,0.5,volkswagen,truck
52001,/c/en/zanella,/r/dbpedia/product,/c/en/motorcycle,0.5,zanella,motorcycle


In [32]:
df_disaster[['start','rel','end','weight']].to_csv('conceptnet_disasters/df-disasters-*.csv',index=False,header=False)

['conceptnet_disasters/df-disasters-00.csv',
 'conceptnet_disasters/df-disasters-01.csv',
 'conceptnet_disasters/df-disasters-02.csv',
 'conceptnet_disasters/df-disasters-03.csv',
 'conceptnet_disasters/df-disasters-04.csv',
 'conceptnet_disasters/df-disasters-05.csv',
 'conceptnet_disasters/df-disasters-06.csv',
 'conceptnet_disasters/df-disasters-07.csv',
 'conceptnet_disasters/df-disasters-08.csv',
 'conceptnet_disasters/df-disasters-09.csv',
 'conceptnet_disasters/df-disasters-10.csv',
 'conceptnet_disasters/df-disasters-11.csv',
 'conceptnet_disasters/df-disasters-12.csv',
 'conceptnet_disasters/df-disasters-13.csv',
 'conceptnet_disasters/df-disasters-14.csv',
 'conceptnet_disasters/df-disasters-15.csv',
 'conceptnet_disasters/df-disasters-16.csv',
 'conceptnet_disasters/df-disasters-17.csv',
 'conceptnet_disasters/df-disasters-18.csv',
 'conceptnet_disasters/df-disasters-19.csv',
 'conceptnet_disasters/df-disasters-20.csv',
 'conceptnet_disasters/df-disasters-21.csv',
 'conceptn

In [30]:
len(df_disaster)

287727

In [33]:
df_da = dd.read_csv('conceptnet_disasters/df-disasters-*.csv',names=['start','rel','end','weight'])

In [34]:
df_da.head()

,start,rel,end,weight
0,/c/en/about,/r/Antonym,/c/en/precisely,0.155
1,/c/en/accident,/r/Antonym,/c/en/on_purpose,0.319
2,/c/en/accident,/r/Antonym,/c/en/planned,0.147
3,/c/en/accident,/r/Antonym,/c/en/purpose,0.319
4,/c/en/accidentally/r,/r/Antonym,/c/en/intentionally,1.000
